In [9]:
import pandas as pd
import numpy as np
import csv
import json
from pandas.io.json import json_normalize

In [10]:
# display only 15 rows of results in each cell
pd.options.display.max_rows = 15
pd.set_option('display.max_columns', 50)

### <font color='orange'>Import Dataframe</font>

Here are the sites explaining the features <br>
1) https://github.com/kevinschaich/billboard <br>
2) https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/ <br>

I'm using the dataset from [Billboard Hot 100 Analytics](https://github.com/RosebudAnwuri/TheArtandScienceofData/blob/master/The%20Making%20of%20Great%20Music/data/music_df.csv) as my primary dataset. However, there is an important feature (song genre/tags) missing from this dataset, so I'm also using the dataset from [Billboard Top 100 Songs](https://github.com/kevinschaich/billboard/tree/master/data/years) to join with my primary dataset.

In [11]:
# this is my primary dataset
primary_df = pd.read_csv('https://raw.githubusercontent.com/RosebudAnwuri/TheArtandScienceofData/master/The%20Making%20of%20Great%20Music/data/music_df.csv')

In [12]:
# define function to generate list of urls for json files of my supplemental dataset
def urls():
    url = []
    url_str = 'https://raw.githubusercontent.com/kevinschaich/billboard/master/data/years/'
    
    for num in range(1950, 2016):
        url_addr = url_str + str(num) + '.json'
        url.append(url_addr)
    return url

In [13]:
# read all json files of my supplemental dataset and concatenate them into one single dataframe
def concat():
    url_list = urls()
    df_from_each_file = (pd.read_json(url) for url in url_list)
    concatenated_df = pd.concat(df_from_each_file, ignore_index=True)
    return concatenated_df

In [14]:
# return the supplemental dataframe
supplemental_df = concat()

In [15]:
# it seems like sentiment and tag features are missing from primary dataset comparing with supplemental dataset
# sentiment column from supplemental dataset are {pos, neu, neg, compound} from primary dataset
# the 'pos' columns are different between primary and supplemental datasets
# the 'pos' column from supplemental dataset probably means the ranking?? But it's not relevant to my analysis so I'm going to ignore it
# I'm going to merge two dataframes and only add the 'tag' as a new feature
print(supplemental_df.columns)
print(primary_df.columns)

Index(['artist', 'difficult_words', 'f_k_grade', 'flesch_index', 'fog_index',
       'lyrics', 'num_dupes', 'num_lines', 'num_syllables', 'num_words', 'pos',
       'sentiment', 'tags', 'title', 'year'],
      dtype='object')
Index(['lyrics', 'num_syllables', 'pos', 'year', 'fog_index', 'flesch_index',
       'num_words', 'num_lines', 'title', 'f_k_grade', 'artist',
       'difficult_words', 'num_dupes', 'neg', 'neu', 'compound', 'id',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'time_signature', 'uri', 'analysis_url',
       'artist_with_features', 'year_bin', 'image', 'cluster', 'Gender'],
      dtype='object')


### <font color='orange'>Merge Dataframe</font>

In [16]:
# for mapping purposes
primary_df['unique_key'] = primary_df['artist'].str.lower().str.replace("'",'').str.replace(' ','').str.replace('-','').str.replace(',','') + '_' + primary_df['title'].str.lower().str.replace("'",'').str.replace(' ','').str.replace('-','').str.replace(',','')
supplemental_df['unique_key'] = supplemental_df['artist'].str.lower().str.replace("'",'').str.replace(' ','').str.replace('-','').str.replace(',','') + '_' + supplemental_df['title'].str.lower().str.replace("'",'').str.replace(' ','').str.replace('-','').str.replace(',','') 

In [17]:
# map the tags from supplemental dataframe to my primary dataframe
mapping = dict(supplemental_df[['unique_key','tags']].values)
primary_df['tags'] = primary_df.unique_key.map(mapping)

In [18]:
# drop the 'unique_key' columns and create a new dataframe called 'df'
df = primary_df.drop('unique_key', axis = 1)

In [19]:
df.to_pickle('music_df.pickle')